In [1]:
import pandas as pd
import numpy as np

In [ ]:
kaggle competitions download -c commonlitreadabilityprize

In [5]:
train_df = pd.read_csv(r"C:\Users\91708\Downloads\CommonLit Readability Prize_data\aa/train.csv")
test_df = pd.read_csv(r"C:\Users\91708\Downloads\CommonLit Readability Prize_data/test.csv")

In [6]:
train_df.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [8]:
test_df.head()

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...


In [9]:
train_df.shape

(2834, 6)

In [10]:
test_df.shape

(7, 4)

In [13]:
train_df.isnull().sum()/len(train_df)*100

id                 0.000000
url_legal         70.712773
license           70.712773
excerpt            0.000000
target             0.000000
standard_error     0.000000
dtype: float64

- drop url_legal and license column 

In [15]:
train_df = train_df.drop(['url_legal','license'], axis=1)
train_df.head()

,id,excerpt,target,standard_error
0,c12129c31,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [18]:
test_df = test_df.drop(['url_legal','license'], axis=1)
test_df.head()

,id,excerpt
0,c0f722661,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,It was a bright and cheerful scene that greete...
3,04caf4e0c,Cell division is the process by which a parent...
4,0e63f8bea,Debugging is the process of finding and resolv...


#### Convert excerpt column into lower case

In [20]:
train_df['excerpt'] = train_df['excerpt'].str.lower()
test_df['excerpt'] = test_df['excerpt'].str.lower()

In [21]:
train_df.head()

,id,excerpt,target,standard_error
0,c12129c31,when the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,"all through dinner time, mrs. fayre was somewh...",-0.315372,0.480805
2,b69ac6792,"as roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,and outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,once upon a time there were three bears who li...,0.247197,0.510845


In [22]:
test_df.head()

,id,excerpt
0,c0f722661,my hope lay in jack's promise that he would ke...
1,f0953f0a5,dotty continued to go to mrs. gray's every nig...
2,0df072751,it was a bright and cheerful scene that greete...
3,04caf4e0c,cell division is the process by which a parent...
4,0e63f8bea,debugging is the process of finding and resolv...


In [23]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

def lemmatize_text_words(words):
    lemmatizer = WordNetLemmatizer()
    
    lemmatized_words = list()
    
    for word in words:
        pos = pos_tag(word)
        
        if pos[0] == 'V':
            pos = wordnet.VERB
        elif pos[0] == 'J':
            pos = wordnet.ADJ
        elif pos[0] == 'R':
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
    
        lemmatized_words.append(lemmatizer.lemmatize(word, pos))
        
    return lemmatized_words

In [24]:
def count_punctuation(words, punct_string='!?.,:'):
    counts = dict.fromkeys(punct_string, 0)
    for word in words:
        if word in punct_string:
            counts[word] += 1
    return counts

In [25]:
def remove_punctuation(words, punct_string='!?.,:'):
    for word in words:
        if word in punct_string:
            words.remove(word)
    return words

In [26]:
from nltk import word_tokenize
from nltk.corpus import stopwords

In [27]:
def make_dictionary(texts):
    words = ' '.join(texts)
    words_dict = set(word_tokenize(words))
    return remove_punctuation(list(words_dict))
    

def prepare_texts(texts, punct_string='!?.,:'):
    stop_words = set(stopwords.words('english'))
    
    prepared_texts = list()
    punctuation_counts = pd.DataFrame(columns=list(punct_string))
    
    for i, text in enumerate(texts):
        text = word_tokenize(text.lower())
        text = [word for word in text if word not in stop_words]
        text = lemmatize_text_words(text)
        punctuation_counts = punctuation_counts.append(pd.Series(count_punctuation(text, punct_string=punct_string)), ignore_index=True)
        text = remove_punctuation(text)
        prepared_texts.append(' '.join(text))
        
    return prepared_texts, punctuation_counts

In [31]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\91708\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [37]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import math

math.sqrt(-np.mean(cross_val_score(GradientBoostingRegressor(), X=train_df, y=train_df.target, scoring='neg_mean_squared_error', n_jobs=-1, verbose=5, cv=4)))


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:   39.6s remaining:   39.6s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   39.6s finished
C:\Users\91708\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
4 fits failed out of a total of 4.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\91708\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\91708\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 486

nan